# MPDP: Visco-elasto-plasticity with isotropic hardening

Associated, rate-dependent plasticity. .

Application of GSM MPDP - Dissipation potential for visco-elasticity included


In [ ]:
%matplotlib widget
from bmcs_matmod.gsm.gsm_mpdp import GSMMPDP
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
sp.init_printing()

### Helmholtz free energy

In [ ]:

E = sp.Symbol(r'E', real=True, nonnegative=True)
eta = sp.Symbol(r'\eta', real=True, nonnegative=True)

mparams = (E, eta)

# ## External state variables

eps = sp.Symbol(r'\varepsilon', real=True)
eps_a = sp.Matrix([eps])
sig = sp.Symbol(r'\sigma', real=True)
sig_a = sp.Matrix([sig])

# ## Internal state variables

eps_v = sp.Symbol(r'\varepsilon^\mathrm{v}', real=True)
eps_v_a = sp.Matrix([eps_v])
sig_v = sp.Symbol(r'\sigma^\mathrm{v}', real=True)
sig_v_a = sp.Matrix([sig_v])

# ## Free energy potential
eps_el = eps - eps_v
U_e_ = sp.Rational(1,2) * E * eps_el**2
F_ = U_e_

Eps_vars = (eps_v_a,)
Sig_vars = (sig_v_a,)
Sig_signs =  (-1,)


### GSM drivers

In [ ]:
gsm_ve = GSMMPDP(
    name = 'gsm_F_1d_mpdp_ve',
    u_vars = eps_a,
    sig_vars = sig_a,
    m_params = mparams,
    Eps_vars = Eps_vars,
    Sig_vars = Sig_vars,
    Sig_signs = Sig_signs,
    F_expr = F_,
    diff_along_rates = False
)

In [ ]:
sig_p = gsm_ve.Sig.as_explicit()[0]

In [ ]:
pi_mech = sp.Rational(1,2) * sig_p**2 / eta
gsm_ve.pi_expr = pi_mech
pi_mech

In [ ]:
(gamma_mech, L_, _, _, dR_dA_n1), (eps_n, delta_eps, Eps_n, delta_A, delta_t, Ox, Ix), Sig_n1, f_n1, R_n1, dR_dA_n1_OI = gsm_ve.Sig_f_R_dR_n1

In [ ]:
L_

In [ ]:
gsm_ve.Sig_

In [ ]:
gsm_ve.phi_

In [ ]:
(gamma_mech, L_, dR_dA_n1), _, Sig_n1, f_n1, R_n1, dR_dA_n1 = gsm_ve.Sig_f_R_dR_n1
gamma_mech

In [ ]:
L_

In [ ]:
Sig_n1

In [ ]:
R_n1_subs = R_n1.replace(sp.sign, lambda x: 1)
R_n1_subs

In [ ]:
dR_dA_n1_subs = dR_dA_n1.replace(sp.sign, lambda x: 1)
dR_dA_n1_subs

In [ ]:
sp.simplify((dR_dA_n1_subs.inv() * R_n1_subs))

In [ ]:
E_val = 30e+3  # Young's modulus in MPa
eta_val = 30e+6  # Viscosity modulus in Pa / s
mparams = (E_val, eta_val)
mparams = (E_val, eta_val)
d_t_val = 0.01 # Time step in seconds
final_time = 20  # Final time in secondsfinal_time = 1

# Generate cyclic sinusoidal loading history
time_t = np.arange(0, final_time, d_t_val)
eps_t_2_cycles = 0.01 * np.sin(2 * np.pi * 1 * time_t / final_time)  # 2 cycles
eps_t_4_cycles = 0.01 * np.sin(2 * np.pi * 4 * time_t / final_time)  # 4 cycles

In [ ]:
fig, ax = plt.subplots(1, 1)
n_I = 1
t_t = time_t
eps_n_t = eps_t_2_cycles
Eps_n_Ib = np.zeros((n_I, gsm_ve.n_Eps_explicit), dtype=np.float64)
t_t, eps_ta_nr_coarse, Eps_t, Sig_t_nr_coarse, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_ve.get_response(eps_n_t, t_t, 100, *mparams)
ax.plot(eps_ta_nr_coarse, Sig_t_nr_coarse[:,:,0], color='blue')
eps_n_t = eps_t_4_cycles
Eps_n_Ib = np.zeros((n_I, gsm_ve.n_Eps_explicit), dtype=np.float64)
t_t, eps_ta_nr_coarse, Eps_t, Sig_t_nr_coarse, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_ve.get_response(eps_n_t, t_t, 100, *mparams)
ax.plot(eps_ta_nr_coarse, Sig_t_nr_coarse[:,:,0], color='red')

In [ ]:
mparams_epd = (1, 1)
eps_n_I = np.array([0, 1.0, 1.0, -1.0, 0, 0], dtype=np.float64)
d_eps_I = np.array([0.1, -0.1, 0.1, -0.1, 2, -2], dtype=np.float64)
n_I = eps_n_I.shape[0]
d_t = 1
Eps_n_Ib = np.zeros((n_I, gsm_ve.n_Eps_explicit), dtype=np.float64)

Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I = gsm_ve.get_state_n1(eps_n_I, d_eps_I, d_t, Eps_n_Ib, 30, *mparams_epd )
Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I